In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category= DeprecationWarning)

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor




In [6]:
def read_data(train_data, test_data):

  train_data = pd.read_csv(train_data)
  test_data = pd.read_csv(test_data)

  combine_df = train_data.append(test_data, ignore_index = True, sort = False)

  return train_data,test_data,combine_df

In [7]:
train_data,test_data,combine_df = read_data('dataset/train.csv','dataset/test.csv')

In [8]:
train_data.head()

,Product_id,Stall_no,instock_date,Market_Category,Customer_name,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price,Selling_Price
0,BRAE2NF6JA5GUEXG,37.0,2015-08-22 18:36:12.000,2,Lillyann,Yes,Fashion,1,68,0.0,376.0,11.0,2983.0,4713.0,4185.947700
1,TUNE8SFB6RJN2HSD,38.0,2016-03-27 21:19:13.000,24,Klynn,Yes,Fashion,0,51,0.0,397.0,12.0,7495.0,10352.0,9271.490256
2,BRAEAR7WZPQGPBZU,9.0,2015-08-18 19:25:22.000,447,Ridge,Yes,Child_care,0,10,0.0,250.0,9.0,5752.0,7309.0,6785.701362
3,WATDZ2ZQ8JPDHCTJ,50.0,2016-03-28 21:53:01.000,23,Abran,Yes,Educational,2,48,0.0,144.0,13.0,5090.0,20814.0,13028.917824
4,JWSEBUKYQPMBZ3RK,7.0,2016-03-29 22:58:53.000,63,Dustyn,Yes,Repair,1,35,1.0,211.0,4.0,2430.0,9261.0,906.553935


In [9]:
train_data.isnull().sum(),train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6368 entries, 0 to 6367
Data columns (total 15 columns):
Product_id          6368 non-null object
Stall_no            6363 non-null float64
instock_date        6368 non-null object
Market_Category     6368 non-null int64
Customer_name       6157 non-null object
Loyalty_customer    6368 non-null object
Product_Category    6368 non-null object
Grade               6368 non-null int64
Demand              6368 non-null int64
Discount_avail      6330 non-null float64
charges_1           6170 non-null float64
charges_2 (%)       6163 non-null float64
Minimum_price       6330 non-null float64
Maximum_price       6025 non-null float64
Selling_Price       6327 non-null float64
dtypes: float64(7), int64(3), object(5)
memory usage: 746.4+ KB


(Product_id            0
 Stall_no              5
 instock_date          0
 Market_Category       0
 Customer_name       211
 Loyalty_customer      0
 Product_Category      0
 Grade                 0
 Demand                0
 Discount_avail       38
 charges_1           198
 charges_2 (%)       205
 Minimum_price        38
 Maximum_price       343
 Selling_Price        41
 dtype: int64,
 None)

In [10]:
test_data.isnull().sum(),test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3430 entries, 0 to 3429
Data columns (total 14 columns):
Product_id          3430 non-null object
Stall_no            3429 non-null float64
instock_date        3430 non-null object
Market_Category     3430 non-null int64
Customer_name       3377 non-null object
Loyalty_customer    3430 non-null object
Product_Category    3430 non-null object
Grade               3430 non-null int64
Demand              3430 non-null int64
Discount_avail      3430 non-null int64
charges_1           3394 non-null float64
charges_2 (%)       3425 non-null float64
Minimum_price       3416 non-null float64
Maximum_price       3430 non-null int64
dtypes: float64(4), int64(5), object(5)
memory usage: 375.3+ KB


(Product_id           0
 Stall_no             1
 instock_date         0
 Market_Category      0
 Customer_name       53
 Loyalty_customer     0
 Product_Category     0
 Grade                0
 Demand               0
 Discount_avail       0
 charges_1           36
 charges_2 (%)        5
 Minimum_price       14
 Maximum_price        0
 dtype: int64,
 None)

In [11]:
train_data.select_dtypes('number').columns

Index(['Stall_no', 'Market_Category', 'Grade', 'Demand', 'Discount_avail',
       'charges_1', 'charges_2 (%)', 'Minimum_price', 'Maximum_price',
       'Selling_Price'],
      dtype='object')

In [12]:
train_data.select_dtypes('object').columns

Index(['Product_id', 'instock_date', 'Customer_name', 'Loyalty_customer',
       'Product_Category'],
      dtype='object')

In [16]:
for col in train_data.columns:
  # print("For", col ,train_data[col].value_counts())
  print("uniquevalues in {} is {}".format(col, len(train_data[col].value_counts())))
  # print(train_data[col].value_counts())
test_productid = test_data.Product_id


uniquevalues in Product_id is 6368
uniquevalues in Stall_no is 50
uniquevalues in instock_date is 5350
uniquevalues in Market_Category is 248
uniquevalues in Customer_name is 5900
uniquevalues in Loyalty_customer is 2
uniquevalues in Product_Category is 10
uniquevalues in Grade is 4
uniquevalues in Demand is 100
uniquevalues in Discount_avail is 2
uniquevalues in charges_1 is 487
uniquevalues in charges_2 (%) is 17
uniquevalues in Minimum_price is 4236
uniquevalues in Maximum_price is 4618
uniquevalues in Selling_Price is 6321


In [17]:
def data_prep(df,df_test):

  df.drop(columns = ['Customer_name','Product_id'],inplace = True)
  df_test.drop(columns = ['Customer_name','Product_id'],inplace = True)

  df = df.dropna(axis=0, subset=['Selling_Price'])
  # df = df.loc(df['Selling_Price']>= 0)
  df.Selling_Price = np.where(df.Selling_Price < 0, 0,df.Selling_Price)
  # df_new = df.loc(df.Selling_Price > 0)
  df = df.dropna(how = 'any')

  # df.Stall_no.fillna(df.Stall_no.median(),inplace=True)
  # df.Discount_avail.fillna(df.Discount_avail.median(),inplace=True)
  # df.charges_1.fillna(df.charges_1.median(),inplace=True)
  # df['charges_2 (%)'].fillna(df['charges_2 (%)'].median(),inplace=True)
  # df.Minimum_price.fillna(df.Minimum_price.median(),inplace=True)
  # df.Maximum_price.fillna(df.Maximum_price.median(),inplace=True)       

  df_test.Stall_no.fillna(df_test.Stall_no.median(),inplace=True)
  df_test.Discount_avail.fillna(df_test.Discount_avail.median(),inplace=True)
  df_test.charges_1.fillna(df_test.charges_1.median(),inplace=True)
  df_test['charges_2 (%)'].fillna(df_test['charges_2 (%)'].median(),inplace=True)
  df_test.Minimum_price.fillna(df_test.Minimum_price.median(),inplace=True)
  df_test.Maximum_price.fillna(df_test.Maximum_price.median(),inplace=True)

  return df,df_test

In [18]:
train_data,test_data = data_prep(train_data,test_data)
train_data.head()

C:\Users\rohit.raj\AppData\Local\Continuum\anaconda3\envs\adapdix\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,Stall_no,instock_date,Market_Category,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price,Selling_Price
0,37.0,2015-08-22 18:36:12.000,2,Yes,Fashion,1,68,0.0,376.0,11.0,2983.0,4713.0,4185.947700
1,38.0,2016-03-27 21:19:13.000,24,Yes,Fashion,0,51,0.0,397.0,12.0,7495.0,10352.0,9271.490256
2,9.0,2015-08-18 19:25:22.000,447,Yes,Child_care,0,10,0.0,250.0,9.0,5752.0,7309.0,6785.701362
3,50.0,2016-03-28 21:53:01.000,23,Yes,Educational,2,48,0.0,144.0,13.0,5090.0,20814.0,13028.917824
4,7.0,2016-03-29 22:58:53.000,63,Yes,Repair,1,35,1.0,211.0,4.0,2430.0,9261.0,906.553935


In [19]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_data['Loyalty_customer'] = le.fit_transform(train_data['Loyalty_customer'])
test_data['Loyalty_customer'] = le.fit_transform(test_data['Loyalty_customer'])

# train_data['Product_Category'] = le.fit_transform(train_data['Product_Category'])


In [20]:
train_data.head()

,Stall_no,instock_date,Market_Category,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price,Selling_Price
0,37.0,2015-08-22 18:36:12.000,2,1,Fashion,1,68,0.0,376.0,11.0,2983.0,4713.0,4185.947700
1,38.0,2016-03-27 21:19:13.000,24,1,Fashion,0,51,0.0,397.0,12.0,7495.0,10352.0,9271.490256
2,9.0,2015-08-18 19:25:22.000,447,1,Child_care,0,10,0.0,250.0,9.0,5752.0,7309.0,6785.701362
3,50.0,2016-03-28 21:53:01.000,23,1,Educational,2,48,0.0,144.0,13.0,5090.0,20814.0,13028.917824
4,7.0,2016-03-29 22:58:53.000,63,1,Repair,1,35,1.0,211.0,4.0,2430.0,9261.0,906.553935


In [21]:
train_data['month_instock'] = pd.to_datetime(train_data['instock_date']).dt.month
train_data['year_instock'] = pd.to_datetime(train_data['instock_date']).dt.year
train_data['day_instock'] = pd.to_datetime(train_data['instock_date']).dt.day
train_data['weekday_instock'] = pd.to_datetime(train_data['instock_date']).dt.weekday


test_data['month_instock'] = pd.to_datetime(test_data['instock_date']).dt.month
test_data['year_instock'] = pd.to_datetime(test_data['instock_date']).dt.year
test_data['day_instock'] = pd.to_datetime(test_data['instock_date']).dt.day
test_data['weekday_instock'] = pd.to_datetime(test_data['instock_date']).dt.weekday

In [22]:
product=pd.get_dummies(train_data[['Product_Category']])
train_data = pd.concat([train_data,product],axis=1)


test_product=pd.get_dummies(test_data[['Product_Category']])
test_data = pd.concat([test_data,test_product],axis=1)

In [23]:
train_data.drop(columns=['Product_Category','instock_date'],inplace=True)

test_data.drop(columns=['Product_Category','instock_date'],inplace=True)

In [24]:
train_data.columns

Index(['Stall_no', 'Market_Category', 'Loyalty_customer', 'Grade', 'Demand',
       'Discount_avail', 'charges_1', 'charges_2 (%)', 'Minimum_price',
       'Maximum_price', 'Selling_Price', 'month_instock', 'year_instock',
       'day_instock', 'weekday_instock', 'Product_Category_Child_care',
       'Product_Category_Cosmetics', 'Product_Category_Educational',
       'Product_Category_Fashion', 'Product_Category_Home_decor',
       'Product_Category_Hospitality', 'Product_Category_Organic',
       'Product_Category_Pet_care', 'Product_Category_Repair',
       'Product_Category_Technology'],
      dtype='object')

In [25]:
test_data.columns

Index(['Stall_no', 'Market_Category', 'Loyalty_customer', 'Grade', 'Demand',
       'Discount_avail', 'charges_1', 'charges_2 (%)', 'Minimum_price',
       'Maximum_price', 'month_instock', 'year_instock', 'day_instock',
       'weekday_instock', 'Product_Category_Child_care',
       'Product_Category_Cosmetics', 'Product_Category_Educational',
       'Product_Category_Fashion', 'Product_Category_Home_decor',
       'Product_Category_Hospitality', 'Product_Category_Organic',
       'Product_Category_Pet_care', 'Product_Category_Repair',
       'Product_Category_Technology'],
      dtype='object')

In [26]:
X = train_data.loc[ :, train_data.columns != 'Selling_Price']
# X.columns
y = train_data['Selling_Price']

In [27]:

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.0001,random_state=42)


In [28]:
# param_grid = [{'n_estimators':np.arange(10,301,10),'max_depth':np.arange(2,20,2),
#                'learning_rate': np.arange(0.01,1.5,0.1),
#                }]
               
#               #  'subsample': np.arange(0.4,1,0.1),'colsample_bytree': (0.4,1,0.1)}]
# xgb = XGBRegressor()
# xgb_grid = GridSearchCV(xgb,param_grid,cv = 5,n_jobs = 15,verbose=6) 
# # xgb_grid = GridSearchCV(xgb,param_grid,cv = 5,n_jobs = 10,verbose=6,scoring='r2') 

# xgb_grid.fit(X_train,y_train)

In [29]:
# xgb_grid.best_params_
# xgb_grid.best_score_

In [30]:
from sklearn.ensemble import RandomForestRegressor


# param_grid = [{'n_estimators':np.arange(50,301,20),
#                'max_depth': np.arange(2,22,2),
#                }]
# rf_reg = RandomForestRegressor()
# reg_grid = GridSearchCV(rf_reg,param_grid,cv = 5,n_jobs = 20) 
# reg_grid.fit(X_train,y_train)

reg = RandomForestRegressor(max_depth = 20, random_state=0,n_estimators = 70,verbose = 1)
reg.fit(X_train,y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed:    5.7s finished


RandomForestRegressor(max_depth=20, n_estimators=70, random_state=0, verbose=1)

In [31]:
y_pred=reg.predict(X_test)
reg.score(X_train,y_train) 

# y_pred=xgb_grid.predict(X_test)
# xgb_grid.score(X_train,y_train) 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed:    0.1s finished


0.998806768110663

In [33]:
# from sklearn.metrics import r2_score
# rsquare=r2_score(y_test,y_pred)
# rsquare

In [34]:
from sklearn.metrics import mean_squared_error as MSE
import numpy as np
 
rmse = np.sqrt(MSE(y_test,y_pred)) 
print("RMSE : % f" %(rmse))



RMSE :  120.105214


In [35]:
test_pred=reg.predict(test_data)
# test_pred=xgb_grid.predict(test_data)

# test_pred = np.abs(test_pred)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed:    0.0s finished


In [36]:
final = pd.DataFrame({'Product_id':test_productid,'Selling_Price':test_pred})
final
# test_productid

,Product_id,Selling_Price
0,SCHE4YSTDVPVZVXW,3262.107193
1,ACCEGCATKHNRXUHW,1920.948140
2,NKCE6GJ5XVJDXNNZ,11788.167354
3,NKCEB8BK3ZXDHDHM,9359.316570
4,TOPEFDXSAHRNPF94,5564.375231
...,...,...
3425,BBAE9K7BXFUTDNBK,3989.713965
3426,RTRDYPA4PJHCJCGF,8878.941577
3427,TUNE8FWQBS7TVVT7,6446.295454
3428,NKCEGMUABCRGUZPE,3558.088837


In [37]:
final.to_csv('sub1.csv',index=False)